In [121]:
! pip freeze | grep -e 'optax' -e 'haiku' -e 'jax' -e 'tensorflow'


dm-haiku==0.0.9
jax==0.3.25
jaxlib==0.3.25
tensorflow==2.11.0
tensorflow-datasets==4.7.0
tensorflow-estimator==2.11.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.11.0


In [122]:
! pip install optax # jax dm-haiku

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 3.3 MB/s eta 0:00:003.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 17.5 MB/s eta 0:00:00


In [49]:
import numpy as np

## Karakter RNN Arany János művein

Az `arany-osszes.txt` file-t az alábbi módon hozhatjuk létre:
```
! wget https://mek.oszk.hu/00500/00597/00597.doc
! libreoffice --cat 00597.doc > arany-osszes.txt
! rm 00597.doc
```

<small><pre>
--2022-12-04 11:35:58--  https://mek.oszk.hu/00500/00597/00597.doc
Resolving mek.oszk.hu (mek.oszk.hu)... 193.6.201.253, 2001:738:0:306::2
Connecting to mek.oszk.hu (mek.oszk.hu)|193.6.201.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4733440 (4,5M) [application/msword]
Saving to: ‘00597.doc’

00597.doc           100%[===================>]   4,51M  2,74MB/s    in 1,7s    

2022-12-04 11:36:00 (2,74 MB/s) - ‘00597.doc’ saved [4733440/4733440]

Warning: failed to launch javaldx - java may not function correctly

(soffice:30311): Gtk-WARNING **: 11:36:01.969: Could not load a pixbuf from icon theme.
This may indicate that pixbuf loaders or the mime database could not be found.
</pre></small>

Colab-on nem érhető el libreoffice. Lehet, hogy lehetne telepíteni, vagy más eszközt használni a szöveg kinyerésére. HF hogyan.

A kurzus github oldalára feltöltöttem az elkészült szöveg file-t, azt fogjuk használni.

In [ ]:
! wget https://github.com/prokaj/elte-python/raw/main/arany-osszes.txt

In [138]:
with open('arany-osszes.txt') as f:
    lines = f.readlines()

print(''.join(lines[:15]+lines[433-15:434]))


ARANY JÁNOS
ÖSSZES KÖLTEMÉNYEI


TARTALOM


Versek
VÁLASZ PETŐFINEK
A VARRÓ LEÁNYOK
A MÉH ROMÁNCA
ARANYAIMHOZ
EGYKORI TANÍTVÁNYOM EMLÉKKÖNYVÉBE
ARCOM VONÁSIT...
ÖLDÖKLŐ ANGYAL
CSANÁD
RÓZSA ÉS IBOLYA II.
KEVEHÁZA
CSABA KIRÁLYFI
CSABA TRILÓGIA (Első dolgozat)
A ZRINYIÁSZ NÉPIES KIDOLGOZÁSA
AZ UTOLSÓ MAGYAR
BUDA HALÁLA
A CSABA-TRILÓGIA ÚJABB ALAPRAJZA
CSABA TRILÓGIA  (Utolsó dolgozat)



Versek
VÁLASZ PETŐFINEK



In [79]:
lines[433]

'VÁLASZ PETŐFINEK\n'

In [82]:
if not lines[0].startswith('VÁLASZ PETŐFINEK'):
    lines = lines[433:]
len(lines), 0.05*len(lines), lines[:10]

(54183,
 2709.15,
 ['VÁLASZ PETŐFINEK\n',
  'Zavarva lelkem, mint a bomlott cimbalom;\n',
  '\xa0\xa0\xa0\xa0\xa0\xa0Örűl a szívem és mégis sajog belé,\n',
  'Hányja veti a hab: mért e nagy jutalom?\n',
  '\xa0\xa0\xa0\xa0\xa0\xa0Petőfit barátul mégsem érdemelé.\n',
  'Hiszen pályadíjul ez nem volt kitűzve...\n',
  '\xa0\xa0\xa0\xa0\xa0\xa0Szerencse, isteni jó szerencse nékem!\n',
  'Máskép szerény művem vetém vala tűzbe,\n',
  '\xa0\xa0\xa0\xa0\xa0\xa0Mert hogyan lett volna nyerni reménységem?\n',
  'És mily sokat nyerék! Pusztán a pályabér\n'])

In [91]:
lineno = 2740
list(enumerate(lines[lineno:lineno+30], lineno))

[(2740, 'Szíves, de bánatos üdvözletem.\n'),
 (2741, 'Repülj utadra most... de nem, ne még,\n'),
 (2742, 'Száritsd föl nedvesült szárnyaidat:\n'),
 (2743, 'Nehéz vagy a könnyhullatás miatt,\n'),
 (2744, 'Mellyel emlékeinek áldozék.\n'),
 (2745, 'Majd csak felszárad a könny... ámde a bú? -\n'),
 (2746, 'Oszlik talán egy hosszu életen. -\n'),
 (2747, 'Vidd asszonyodnak, egyszerű lapocska,\n'),
 (2748, 'Szíves, de bánatos üdvözletem.\n'),
 (2749, '(1850)\n'),
 (2750, 'FURKÓ TAMÁS\n'),
 (2751, 'Ki zengi meg, ha én nem, a mult nagy napjait:\n'),
 (2752, 'Vitéz Furkó Tamásnak kegyetlen dolgait,\n'),
 (2753, 'Amelyeket viselt... vagy, mondani akarom:\n'),
 (2754, 'Mik őt úgy megviselték, vizen és szárazon.\n'),
 (2755, 'Mindig bolond hónap volt, mióta e világ,\n'),
 (2756, 'Az, melyet márciusnak keresztelt a diák:\n'),
 (2757, 'Fagy, hó, eső, derült ég, vihar... mind egy napon!\n'),
 (2758, 'Csak úgy bámul belé a bölcs kalendárium.\n'),
 (2759, 'De amióta Caesart hatalma nimbusán\n'),
 (2760,

In [92]:
test_lines = lines[:2750]
train_lines = lines[2750:]

print(''.join(test_lines[-10:]))
print('-'*100)
print(''.join(train_lines[:10]))

Szíves, de bánatos üdvözletem.
Repülj utadra most... de nem, ne még,
Száritsd föl nedvesült szárnyaidat:
Nehéz vagy a könnyhullatás miatt,
Mellyel emlékeinek áldozék.
Majd csak felszárad a könny... ámde a bú? -
Oszlik talán egy hosszu életen. -
Vidd asszonyodnak, egyszerű lapocska,
Szíves, de bánatos üdvözletem.
(1850)

----------------------------------------------------------------------------------------------------
FURKÓ TAMÁS
Ki zengi meg, ha én nem, a mult nagy napjait:
Vitéz Furkó Tamásnak kegyetlen dolgait,
Amelyeket viselt... vagy, mondani akarom:
Mik őt úgy megviselték, vizen és szárazon.
Mindig bolond hónap volt, mióta e világ,
Az, melyet márciusnak keresztelt a diák:
Fagy, hó, eső, derült ég, vihar... mind egy napon!
Csak úgy bámul belé a bölcs kalendárium.
De amióta Caesart hatalma nimbusán



In [93]:
from itertools import chain

In [94]:
from collections import Counter
cnt_char = Counter(chain.from_iterable(lines))

In [95]:
print(''.join(sorted(cnt_char.keys())))
print(cnt_char)


 !()*,-.0123456789:;<=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz| «­»ÁÂÉÍÓÖÚÜáäéëíîóôöúüŐőŰű‘’”„
Counter({' ': 233976, 'e': 137532, 'a': 122239, 't': 100108, 'n': 88813, 'l': 86906, 's': 77114, 'r': 60215, 'k': 57595, 'o': 56734, 'i': 56391, 'z': 54694, '\n': 54183, 'g': 51262, 'm': 50227, 'á': 50094, 'é': 43020, ',': 41929, 'y': 39060, 'd': 35166, 'v': 29797, 'b': 26781, 'h': 23873, 'j': 21762, 'ö': 17063, '.': 15713, '\xa0': 14914, 'u': 13193, 'f': 12964, 'p': 12613, 'ő': 11373, 'ó': 10815, 'c': 10704, 'M': 7812, '-': 7599, 'ü': 7326, ':': 7232, 'S': 5754, 'í': 5701, 'A': 5659, ';': 5513, 'H': 5095, 'ú': 4941, '!': 4363, 'E': 3954, 'K': 3904, 'N': 3619, '„': 3469, 'ű': 3030, 'T': 2753, '”': 2542, 'V': 2435, '?': 2378, 'D': 2361, 'B': 2326, 'I': 2316, 'L': 2178, 'F': 1871, 'É': 1763, '1': 1586, 'R': 1399, 'O': 1394, '’': 1289, 'C': 1263, 'P': 1127, 'J': 1115, ')': 989, '(': 987, 'G': 984, '8': 912, '2': 848, 'Á': 778, '5': 686, '3': 617, '7': 604, '4

`\xad` a soft-hyphen karakter, `xa0` non-breakable space, `\uf0..` csak egy helyen használatos a hexaméterek ritmusának leírására. 

In [96]:
letters = [k for k in cnt_char if ord(k)< int('f000', 16) and k not in '\xa0\xad']
print(letters)
int2char = dict(enumerate(letters, 1))
char2int = {k: i for i, k in int2char.items()}
char2int['\xa0'] = char2int[' ']

['V', 'Á', 'L', 'A', 'S', 'Z', ' ', 'P', 'E', 'T', 'Ő', 'F', 'I', 'N', 'K', '\n', 'a', 'v', 'r', 'l', 'e', 'k', 'm', ',', 'i', 'n', 't', 'b', 'o', 'c', ';', 'Ö', 'ű', 's', 'z', 'í', 'é', 'g', 'j', 'H', 'á', 'y', 'h', ':', 'u', '?', 'ő', 'f', 'd', '.', 'p', 'ó', '!', 'M', 'É', 'D', 'ö', 'O', '-', 'J', 'ú', 'ü', '(', '1', '8', '4', '7', ')', 'R', 'Ó', 'Y', 'B', 'G', 'C', '’', 'U', '„', '”', 'Ü', 'Í', '2', 'Ú', '3', 'w', 'x', 'X', '6', '5', '9', '*', '0', '=', 'Q', 'ë', '‘', 'Ű', 'ä', 'W', 'î', 'q', 'Â', 'ô', '»', '«', '|', '<', '[', ']', '_']


In [112]:
def text2int_array(txt):
    result = []
    for x in txt:
        code = char2int.get(x, -1)
        if code>-1:
            result.append(code)
    return np.array(result, dtype=np.int32)

def int_array2str(arr: np.ndarray):
    return ''.join(int2char[x] for x in arr)
     


In [113]:
#train_data = text2int_array(chain.from_iterable(train_lines))
test_data = text2int_array(chain.from_iterable(test_lines))

In [114]:
print(lines[:5])
int_array2str(test_data[:100])

['VÁLASZ PETŐFINEK\n', 'Zavarva lelkem, mint a bomlott cimbalom;\n', '\xa0\xa0\xa0\xa0\xa0\xa0Örűl a szívem és mégis sajog belé,\n', 'Hányja veti a hab: mért e nagy jutalom?\n', '\xa0\xa0\xa0\xa0\xa0\xa0Petőfit barátul mégsem érdemelé.\n']


'VÁLASZ PETŐFINEK\nZavarva lelkem, mint a bomlott cimbalom;\n      Örűl a szívem és mégis sajog belé,\nH'

In [115]:
import tensorflow.data as tf_data
from tensorflow.nest import map_structure as tf_map_structure
from tensorflow import transpose as tf_transpose

In [135]:
def make_ds(
    lines,
    sequence_len=128, 
    batch_size=32
):
    encoded_text = text2int_array(chain.from_iterable(lines))

    ds = tf_data.Dataset.from_tensors({'text': encoded_text})
    ds = ds.map(lambda x: {'input': x['text'][:-1], 'target': x['text'][1:]})
    ds = ds.unbatch()
    ds = ds.batch(sequence_len, drop_remainder=True)
    ds = ds.shuffle(100)
    ds = ds.repeat()
    ds = ds.batch(batch_size=batch_size)
    ds = ds.map(lambda b: tf_map_structure(tf_transpose, b))  # Time major

    return ds.as_numpy_iterator()

NUM_CHARS = max(int2char)


In [136]:
NUM_CHARS, min(int2char)

(109, 1)

In [129]:

test_ds = make_ds(test_lines)

In [130]:

batch = next(train_ds)

In [131]:
batch

{'input': array([[42, 27, 34, ...,  7, 26, 50],
        [28, 25, 35, ..., 38, 27, 50],
        [21,  7, 21, ..., 42, 21, 16],
        ...,
        [26, 36, 20, ..., 21, 20, 19],
        [49, 18, 42, ..., 23, 21, 24],
        [21, 21, 21, ..., 21, 26, 16]], dtype=int32),
 'target': array([[28, 25, 35, ..., 38, 27, 50],
        [21,  7, 21, ..., 42, 21, 16],
        [26, 21, 19, ..., 21,  7, 63],
        ...,
        [49, 18, 42, ..., 23, 21, 24],
        [21, 21, 21, ..., 21, 26, 16],
        [26,  7, 27, ..., 27,  7, 72]], dtype=int32)}

In [137]:
# Copyright 2020 DeepMind Technologies Limited. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================
"""Character-level language modelling with a recurrent network in JAX."""

from typing import Any, NamedTuple

from absl import logging
from absl import flags

import haiku as hk

import jax
from jax import lax
import jax.numpy as jnp
import numpy as np
import optax

if not hasattr(flags.FLAGS, 'train_batch_size'):
    TRAIN_BATCH_SIZE = flags.DEFINE_integer('train_batch_size', 32, '')
    EVAL_BATCH_SIZE = flags.DEFINE_integer('eval_batch_size', 1000, '')
    SEQUENCE_LENGTH = flags.DEFINE_integer('sequence_length', 128, '')
    HIDDEN_SIZE = flags.DEFINE_integer('hidden_size', 256, '')
    SAMPLE_LENGTH = flags.DEFINE_integer('sample_length', 128, '')
    LEARNING_RATE = flags.DEFINE_float('learning_rate', 1e-3, '')
    TRAINING_STEPS = flags.DEFINE_integer('training_steps', 50_000, '')
    EVALUATION_INTERVAL = flags.DEFINE_integer('evaluation_interval', 1000, '')
    SAMPLING_INTERVAL = flags.DEFINE_integer('sampling_interval', 1000, '')
    SEED = flags.DEFINE_integer('seed', 42, '')



In [124]:
from absl import app


In [125]:
app.parse_flags_with_usage(['main', '-v', '1'])
#flags.ArgumentParser().parse('')
TRAIN_BATCH_SIZE.value

32

In [126]:
flags.FLAGS.flag_values_dict()

{'logtostderr': False,
 'alsologtostderr': False,
 'log_dir': '',
 'v': 1,
 'verbosity': 1,
 'logger_levels': {},
 'stderrthreshold': 'fatal',
 'showprefixforinfo': True,
 'run_with_pdb': False,
 'pdb_post_mortem': False,
 'pdb': False,
 'run_with_profiling': False,
 'profile_file': None,
 'use_cprofile_for_profiling': True,
 'only_check_args': False,
 'op_conversion_fallback_to_while_loop': True,
 'runtime_oom_exit': True,
 'hbm_oom_exit': True,
 'delta_threshold': 0.5,
 'tt_check_filter': False,
 'tt_single_core_summaries': False,
 'test_srcdir': '',
 'test_tmpdir': '/tmp/absl_testing',
 'test_random_seed': 301,
 'test_randomize_ordering_seed': '',
 'xml_output_file': '',
 'chex_n_cpu_devices': 1,
 'chex_assert_multiple_cpu_devices': False,
 'chex_skip_pmap_variant_if_single_device': True,
 'train_batch_size': 32,
 'eval_batch_size': 1000,
 'sequence_length': 128,
 'hidden_size': 256,
 'sample_length': 128,
 'learning_rate': 0.001,
 'training_steps': 50000,
 'evaluation_interval': 10

In [ ]:


class LoopValues(NamedTuple):
  tokens: jnp.ndarray
  state: Any
  rng_key: jnp.ndarray


class TrainingState(NamedTuple):
  params: hk.Params
  opt_state: optax.OptState


def make_network() -> hk.RNNCore:
  """Defines the network architecture."""
  model = hk.DeepRNN([
      lambda x: jax.nn.one_hot(x, num_classes=NUM_CHARS),
      hk.LSTM(HIDDEN_SIZE.value),
      jax.nn.relu,
      hk.LSTM(HIDDEN_SIZE.value),
      hk.nets.MLP([HIDDEN_SIZE.value, NUM_CHARS]),
  ])
  return model


def make_optimizer() -> optax.GradientTransformation:
  """Defines the optimizer."""
  return optax.adam(LEARNING_RATE.value)


def sequence_loss(batch: Batch) -> jnp.ndarray:
  """Unrolls the network over a sequence of inputs & targets, gets loss."""
  # Note: this function is impure; we hk.transform() it below.
  core = make_network()
  sequence_length, batch_size = batch['input'].shape
  initial_state = core.initial_state(batch_size)
  logits, _ = hk.dynamic_unroll(core, batch['input'], initial_state)
  log_probs = jax.nn.log_softmax(logits)
  one_hot_labels = jax.nn.one_hot(batch['target'], num_classes=logits.shape[-1])
  return -jnp.sum(one_hot_labels * log_probs) / (sequence_length * batch_size)


@jax.jit
def update(state: TrainingState, batch: Batch) -> TrainingState:
  """Does a step of SGD given inputs & targets."""
  _, optimizer = make_optimizer()
  _, loss_fn = hk.without_apply_rng(hk.transform(sequence_loss))
  gradients = jax.grad(loss_fn)(state.params, batch)
  updates, new_opt_state = optimizer(gradients, state.opt_state)
  new_params = optax.apply_updates(state.params, updates)
  return TrainingState(params=new_params, opt_state=new_opt_state)


def sample(
    rng_key: jnp.ndarray,
    context: jnp.ndarray,
    sample_length: int,
) -> jnp.ndarray:
  """Draws samples from the model, given an initial context."""
  # Note: this function is impure; we hk.transform() it below.
  assert context.ndim == 1  # No batching for now.
  core = make_network()

  def body_fn(t: int, v: LoopValues) -> LoopValues:
    token = v.tokens[t]
    next_logits, next_state = core(token, v.state)
    key, subkey = jax.random.split(v.rng_key)
    next_token = jax.random.categorical(subkey, next_logits, axis=-1)
    new_tokens = v.tokens.at[t + 1].set(next_token)
    return LoopValues(tokens=new_tokens, state=next_state, rng_key=key)

  logits, state = hk.dynamic_unroll(core, context, core.initial_state(None))
  key, subkey = jax.random.split(rng_key)
  first_token = jax.random.categorical(subkey, logits[-1])
  tokens = jnp.zeros(sample_length, dtype=np.int32)
  tokens = tokens.at[0].set(first_token)
  initial_values = LoopValues(tokens=tokens, state=state, rng_key=key)
  values: LoopValues = lax.fori_loop(0, sample_length, body_fn, initial_values)

  return values.tokens


def main():
  flags.FLAGS.alsologtostderr = True
  flags.FLAGS.verbosity = 1

  # Make training dataset.
  train_data = make_ds(
      train_lines,
      batch_size=TRAIN_BATCH_SIZE.value,
      sequence_len=SEQUENCE_LENGTH.value
    )

  # Make evaluation dataset(s).
  eval_data = {  # pylint: disable=g-complex-comprehension
      'train': make_ds(
        train_lines,
        batch_size=EVAL_BATCH_SIZE.value,
        sequence_len=SEQUENCE_LENGTH.value
      ),
      'test': make_ds(
        test_lines,
        batch_size=EVAL_BATCH_SIZE.value,
        sequence_len=SEQUENCE_LENGTH.value
      )
  }

  # Make loss, sampler, and optimizer.
  params_init, loss_fn = hk.without_apply_rng(hk.transform(sequence_loss))
  _, sample_fn = hk.without_apply_rng(hk.transform(sample))
  opt_init, _ = make_optimizer()

  loss_fn = jax.jit(loss_fn)
  sample_fn = jax.jit(sample_fn, static_argnums=[3])

  # Initialize training state.
  rng = hk.PRNGSequence(SEED.value)
  initial_params = params_init(next(rng), next(train_data))
  initial_opt_state = opt_init(initial_params)
  state = TrainingState(params=initial_params, opt_state=initial_opt_state)

  # Training loop.
  for step in range(TRAINING_STEPS.value + 1):
    # Do a batch of SGD.
    train_batch = next(train_data)
    state = update(state, train_batch)

    # Periodically generate samples.
    if step % SAMPLING_INTERVAL.value == 0:
      context = train_batch['input'][:, 0]  # First element of training batch.
      assert context.ndim == 1
      rng_key = next(rng)
      samples = sample_fn(state.params, rng_key, context, SAMPLE_LENGTH.value)

      prompt = decode(context)
      continuation = decode(samples)

      logging.info('\n===Prompt:\n%s\n===\n', prompt)
      logging.info('\n---Continuation:\n%s\n---\n', continuation)

    # Periodically evaluate training and test loss.
    if step % EVALUATION_INTERVAL.value == 0:
      for split, ds in eval_data.items():
        eval_batch = next(ds)
        loss = loss_fn(state.params, eval_batch)
        logging.info({
            'step': step,
            'loss': float(loss),
            'split': split,
        })


In [ ]:
main()